# Chapter 12 - Behavioral Patterns

Template Method and Chain of Responsibility patterns.

## The Template Method pattern

Base class defines the outline of an algorithm while leaving parts open for modification by sub-classes.

In [39]:
classDiagram
    direction BT
    class AbstractClass {
        +TemplateMethod()
        #Operation()
    }
    class Client {

    }
    class ConcreteClass {
        #Operation()
    }
   Client o-- AbstractClass : uses
   ConcreteClass ..|> AbstractClass

In [40]:
public abstract class SearchMachine
{
    protected int[] Values { get; }

    protected SearchMachine(params int[] values)
    {
        Values = values ?? throw new ArgumentNullException(nameof(values));
    }

    public int? IndexOf(int value)
    {
        if (Values.Length == 0) { return null; }
        var result = Find(value);
        return result;
    }
    protected abstract int? Find(int value);
}

public class BinarySearchMachine : SearchMachine
{
    public BinarySearchMachine(params int[] values)
        : base(values.OrderBy(v => v).ToArray()) { }

    protected override int? Find(int value)
    {
        var index = Array.BinarySearch(Values, value);
        return index < 0 ? null : index;
    }
}

public class LinearSearchMachine : SearchMachine
{
    public LinearSearchMachine(params int[] values) : base(values) { }

    protected override int? Find(int value)
    {
        for (var i = 0; i < Values.Length; i++)
        {
            if (Values[i] == value) { return i; }
        }
        return null;
    }
}

var linearSearch = new LinearSearchMachine(1, 10, 5, 2, 123, 333, 4);
var binarySearch = new BinarySearchMachine(1, 2, 3, 4, 5, 6, 7, 8, 9, 10);

Console.WriteLine(linearSearch.IndexOf(123)); // 4
Console.WriteLine(linearSearch.IndexOf(11)); // null

Console.WriteLine(binarySearch.IndexOf(9)); // 8
Console.WriteLine(binarySearch.IndexOf(11)); // null


4


8
8




```csharp
builder.Services
    .AddSingleton<SearchMachine>(x
    => new LinearSearchMachine(1, 10, 5, 2, 123, 333, 4))
    .AddSingleton<SearchMachine>(x
    => new BinarySearchMachine(1, 2, 3, 4, 5, 6, 7, 8, 9, 10);
 ```

```csharp
IEnumerable<SearchResult> SearchForIndex(int number, IEnumerable<SearchMachine> searchMachines)
{
    foreach (var searchMachine in searchMachines)
    {
        var name = searchMachine.GetType().Name;
        var index = searchMachine.IndexOf(number);
        var found = index.HasValue;
        yield return new SearchResult(number, name, found, index);
    }
}
public record class SearchResult(
    int SearchedNumber,
    string Name,
    bool Found,
    int? Index
);
```

In [41]:

int size = 1000000; // Number of unique integers needed
int min = 0;
int max = 1000000; // Must be at least size

// Generate a range, shuffle, and take the first 'size' elements
Random rand = new Random();
int[] uniqueRandomArray = Enumerable.Range(min, max - min)
    .OrderBy(_ => rand.Next())
    .Take(size)
    .ToArray();

var linearSearch = new LinearSearchMachine(uniqueRandomArray);
var binarySearch = new BinarySearchMachine(uniqueRandomArray);


In [42]:
#!time
Console.WriteLine(linearSearch.IndexOf(1000));

300029


Wall time: 11.7086ms

In [43]:
#!time
Console.WriteLine(binarySearch.IndexOf(1000));

1000


Wall time: 9.0354ms

## The Chain of Responsibility Pattern

- Chain multiple handlers that each solve a limited problem
- Each handler is independent and passes the request to the next when complete
- The terminal handler is the last in the chain
- Can divide complex logic into smaller testable chunks

In [46]:
classDiagram
    direction TB
    class IHandler {
        <<interface>>
        +HandleRequest(request)
    }
    class HandlerA {
        -next: IHandler
        +HandleRequest(request)
    }
    class HandlerB {
        -next: IHandler
        +HandleRequest(request)
    }
    class HandlerC {
        +HandleRequest(request)
    }
    class Client {

    }
    IHandler <|-- HandlerA
    IHandler <|-- HandlerB
    IHandler <|-- HandlerC
    Client o-- IHandler : uses

### Project: Message interpreter

In [52]:
public record class Message(string Name, string? Payload);

public interface IMessageHandler
{
    void Handle(Message message);
}

public class AlarmPausedHandler : IMessageHandler
{
    private readonly IMessageHandler? _next;
    public AlarmPausedHandler(IMessageHandler? next = null)
    {
        _next = next;
    }

    public void Handle(Message message)
    {
        if (message.Name == "AlarmPaused")
        {
            // Do something clever with the Payload
        }
        else
        {
            _next?.Handle(message);
        }
    }
}

public class AlarmStoppedHandler : IMessageHandler
{
    private readonly IMessageHandler? _next;
    public AlarmStoppedHandler(IMessageHandler? next = null)
    {
        _next = next;
    }

    public void Handle(Message message)
    {
        if (message.Name == "AlarmStopped")
        {
            // Do something clever with the Payload
        }
        else
        {
            _next?.Handle(message);
        }
    }
}

public class AlarmTriggeredHandler : IMessageHandler
{
    private readonly IMessageHandler? _next;
    public AlarmTriggeredHandler(IMessageHandler? next = null)
    {
        _next = next;
    }

    public void Handle(Message message)
    {
        if (message.Name == "AlarmTriggered")
        {
            // Do something clever with the Payload
        }
        else
        {
            _next?.Handle(message);
        }
    }
}

public class DefaultHandler : IMessageHandler
{
    public void Handle(Message message)
    {
        throw new NotSupportedException(
            $"Messages named '{message.Name}' are not supported.");
    }
}


public string HandleMessage(Message message)
{
    
    var messageHandler = new AlarmTriggeredHandler(
            new AlarmPausedHandler(
                new AlarmStoppedHandler(
                    new DefaultHandler()
                )));
    try
    {
        messageHandler.Handle(message);
        return $"Message '{message.Name}' handled successfully.";
    }
    catch (NotSupportedException ex)
    {
        return ex.Message;
    }
}



HandleMessage(new Message("AlarmPaused", "Payload1")).Display();
HandleMessage(new Message("AlarmStopped", "Payload2")).Display();
HandleMessage(new Message("Notsupported", "Payload3")).Display();


Message 'AlarmPaused' handled successfully.

Message 'AlarmStopped' handled successfully.

Messages named 'Notsupported' are not supported.

See [C12\src\ImprovedChainOfResponsibility](../C12/src/ImprovedChainOfResponsibility/Program.cs) for an example of combining the template method with chain of responsibility. Also [C12\src\FinalChainOfResponsibility](../C12/src/FinalChainOfResponsibility/Program.cs) then improves it.